<a href="https://colab.research.google.com/github/lazysimon/NYCDSA_CapstoneProject/blob/master/movie_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
from urllib.request import urlopen
from numpy import loadtxt

In [0]:
def read_the_dat(filepath):
    with urlopen(filepath) as raw_data:
        datfileL=pd.DataFrame(l.decode("utf-8").rstrip().split('::') for l in raw_data)
    return datfileL

In [0]:
movie_fname = 'https://raw.githubusercontent.com/lazysimon/NYCDSA_CapstoneProject/master/latest/movies.dat'
movie_df = read_the_dat(movie_fname)

In [0]:
rating_fname = 'https://raw.githubusercontent.com/lazysimon/NYCDSA_CapstoneProject/master/latest/ratings.dat'
rating_df = read_the_dat(rating_fname)
rating_df.columns = ['userID', 'movieID', 'rating', 'timestamp']
rating_df.head()

In [0]:
data = pd.DataFrame(raw_data)

In [42]:
data[0][0].decode("utf-8").rstrip().split('::')

['0000008',
 'Edison Kinetoscopic Record of a Sneeze (1894)',
 'Documentary|Short']

In [52]:
rating_df.columns = ['userID', 'movieID', 'rating', 'timestamp']
rating_df.head()
#rating_df.columns

,userID,movieID,rating,timestamp
0,1,0111161,10,1373234211
1,1,0117060,7,1373415231
2,1,0120755,6,1373424360
3,1,0317919,6,1373495763
4,1,0454876,10,1373621125


In [53]:
movie_df.head()

,0,1,2
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,
4,0000091,Le manoir du diable (1896),Short|Horror
